# BT4222
## Data Processing



In [1]:
pip install spacytextblob --quiet

     |████████████████████████████████| 6.0 MB 8.6 MB/s 
     |████████████████████████████████| 10.1 MB 55.1 MB/s 
     |████████████████████████████████| 181 kB 47.9 MB/s 
     |████████████████████████████████| 42 kB 1.4 MB/s 
     |████████████████████████████████| 628 kB 47.2 MB/s 
     |████████████████████████████████| 451 kB 53.1 MB/s 


In [2]:
!python -m spacy download en_core_web_sm --quiet

     |████████████████████████████████| 13.9 MB 85 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [3]:
# Imports
import pandas as pd
import numpy as np
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob
en = spacy.load('en_core_web_sm')
en.add_pipe('spacytextblob')

In [4]:
# Read raw reviews
# df = pd.read_csv("scraped_output.csv")
df = pd.read_csv("reg_scraped_output.csv")
df.dropna(inplace=True)
df.head(3)

,Review,Date,Stars,Restaurant
0,There are lots of Indian restaurant at Tanjong...,4 days ago,5 stars,Gravy_Restaurant
2,"The dal makhni was sweet, seemed like sugar wa...",a week ago,2 stars,Gravy_Restaurant
4,We are regular customers - the food is amazing...,2 weeks ago,5 stars,Gravy_Restaurant


### Feature-Descriptor Extraction

In [5]:
# Function to break sentence into constituents
# Function to do feature-descriptor extraction from each row:
def get_feature_descriptors(text):
#     text = row["text"]

    doc = en(text)
#     deplacy.render(doc)
    seen = set() # keep track of covered words
    dict_noun = {} # This is for the dict of Noun - Adverb, adj, verb
    chunks = []
    for sent in doc.sents:
    #calculate the dict_noun first

        for word in sent:
                
                if word.pos_ == 'NOUN':
                    current_noun = word.text
                    dict_noun[current_noun] = []
                if word.pos_ == 'ADV' or word.pos_ == 'ADJ' or word.pos_ == 'VERB':
                    try:
                        dict_noun[current_noun].append(word.text)
                    except:
                        pass;
                    

        #See if there are conjunctions to split on. If so get the heads for the subtrees
        heads = [cc for cc in sent if cc.dep_ == 'conj']
#         print(heads)

        # If there are no conjunctions split only on punctuation.
        if(len(heads)==0):
            counter = 0
            array = []
            for word in sent:
                if word.pos_ != 'PUNCT':
                    array.append(word)
                    continue
                chunk = (' '.join([ww.text for ww in array]))
                chunks.append( (counter, chunk) )
                counter = counter + 1
                array = []
            chunk = (' '.join([ww.text for ww in array]))
            chunks.append( (counter, chunk) )
        else:
            for head in heads:
                words = [None]*100
                counter = 0
                for i in head.subtree:
                    if(i in heads):
                        if(i != head):
                            break
                    if(i.pos_ == 'PUNCT'):
                        break
                    words[counter] = i
                    counter = counter + 1
                    if(i.dep_ == 'cc'):
                        break
                res = []
                for val in words:
                    if val != None :
                        res.append(val)
                for word in res:
                    seen.add(word)
                chunk = (' '.join([ww.text for ww in res]))
                chunks.append( (head.i, chunk) )
            unseen = [ww for ww in sent if ww not in seen]
            counter = 0
            array = []
            for word in unseen:
                if word.pos_ != 'PUNCT':
                    array.append(word)
                    continue
                chunk = (' '.join([ww.text for ww in array if ww.dep_ != 'cc']))
                chunks.append( (counter, chunk) )
                counter = counter + 1
                array = []
            chunk = (' '.join([ww.text for ww in array if ww.dep_ != 'cc']))
            chunks.append( (counter, chunk) )
    return [dict_noun]

In [6]:
# Apply function to get feature-descriptors
df["feature_descriptors"] = df["Review"].apply(lambda x:get_feature_descriptors(x))

In [7]:
df.head(1)

,Review,Date,Stars,Restaurant,feature_descriptors
0,There are lots of Indian restaurant at Tanjong...,4 days ago,5 stars,Gravy_Restaurant,"[{'lots': ['Indian'], 'restaurant': [], 'area'..."


In [8]:
def clause_level(text):
  doc = en(text)
#     deplacy.render(doc)
  seen = set() # keep track of covered words
  dict_noun = {} # This is for the dict of Noun - Adverb, adj, verb
  chunks = []
  for sent in doc.sents:
  #calculate the dict_noun first

      # for word in sent:
              
      #         if word.pos_ == 'NOUN':
      #             current_noun = word.text
      #             dict_noun[current_noun] = []
      #         if word.pos_ == 'ADV' or word.pos_ == 'ADJ' or word.pos_ == 'VERB':
      #             try:
      #                 dict_noun[current_noun].append(word.text)
      #             except:
      #                 pass;
                  

      #See if there are conjunctions to split on. If so get the heads for the subtrees
      heads = [cc for cc in sent if cc.dep_ == 'conj']
#         print(heads)

      # If there are no conjunctions split only on punctuation.
      if(len(heads)==0):
          counter = 0
          array = []
          for word in sent:
              if word.pos_ != 'PUNCT':
                  array.append(word)
                  continue
              chunk = (' '.join([ww.text for ww in array]))
              chunks.append( (counter, chunk) )
              counter = counter + 1
              array = []
          chunk = (' '.join([ww.text for ww in array]))
          chunks.append( (counter, chunk) )
      else:
          for head in heads:
              words = [None]*100
              counter = 0
              for i in head.subtree:
                  if(i in heads):
                      if(i != head):
                          break
                  if(i.pos_ == 'PUNCT'):
                      break
                  words[counter] = i
                  counter = counter + 1
                  if(i.dep_ == 'cc'):
                      break
              res = []
              for val in words:
                  if val != None :
                      res.append(val)
              for word in res:
                  seen.add(word)
              chunk = (' '.join([ww.text for ww in res]))
              chunks.append( (head.i, chunk) )
          unseen = [ww for ww in sent if ww not in seen]
          counter = 0
          array = []
          for word in unseen:
              if word.pos_ != 'PUNCT':
                  array.append(word)
                  continue
              chunk = (' '.join([ww.text for ww in array if ww.dep_ != 'cc']))
              chunks.append( (counter, chunk) )
              counter = counter + 1
              array = []
          chunk = (' '.join([ww.text for ww in array if ww.dep_ != 'cc']))

          chunks.append( (counter, chunk) )
  chunks = sorted(chunks, key=lambda x: x[0])
  # print(dict_noun)

  retdict = dict()

  for ii, chunk in chunks:
      doc = en(chunk)
      for i in doc.sents:
          for cc in i:
              if cc.pos_ == 'NOUN':
                  # print("Noun :" + cc.text)
                  retdict[cc.text] = chunk
  return [retdict]

In [9]:
df["clause_level_breakdown"] = df["Review"].apply(lambda x: clause_level(x))

In [10]:
df.head(3)

,Review,Date,Stars,Restaurant,feature_descriptors,clause_level_breakdown
0,There are lots of Indian restaurant at Tanjong...,4 days ago,5 stars,Gravy_Restaurant,"[{'lots': ['Indian'], 'restaurant': [], 'area'...",[{'lots': 'There are lots of Indian restaurant...
2,"The dal makhni was sweet, seemed like sugar wa...",a week ago,2 stars,Gravy_Restaurant,"[{'dal': [], 'makhni': ['sweet', 'seemed'], 's...","[{'dal': 'The dal makhni was sweet', 'makhni':..."
4,We are regular customers - the food is amazing...,2 weeks ago,5 stars,Gravy_Restaurant,"[{'customers': [], 'food': ['had']}]","[{'customers': 'We are regular customers', 'fo..."


### Word Embeddings

In [11]:
!wget http://nlp.stanford.edu/data/glove.6B.zip --quiet
!unzip -q glove.6B.zip

In [12]:
embeddings_index = {}
with open('glove.6B.100d.txt') as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

Found 400000 word vectors.


In [13]:
def add_vectors(feats):
  feats = feats[0]

  newdict = dict()
  for key in list(feats.keys()):
    list_of_desc = feats[key]
    vect_list = []
    for word in list_of_desc:
      try:
        vect_list.append(embeddings_index[word])
      except:
        pass
    newdict[key] = vect_list
  return [newdict]

In [14]:
df["vector_descriptors"] = df["feature_descriptors"].apply(lambda x: add_vectors(x))

In [15]:
df.head(1)

,Review,Date,Stars,Restaurant,feature_descriptors,clause_level_breakdown,vector_descriptors
0,There are lots of Indian restaurant at Tanjong...,4 days ago,5 stars,Gravy_Restaurant,"[{'lots': ['Indian'], 'restaurant': [], 'area'...",[{'lots': 'There are lots of Indian restaurant...,"[{'lots': [], 'restaurant': [], 'area': [[-0.0..."


In [16]:
def add_clause_level_vectors(feats):
  feats = feats[0]

  newdict = dict()
  for feat in feats.keys():
    clause = feats[feat]
    # print(clause, type(clause))
    vect_list = []
    for word in clause.split(' '):
      # print(word)
      try:
        vect_list.append(embeddings_index[word])
      except:
        pass
    newdict[feat] = vect_list
  return [newdict]

In [17]:
# print(df["clause_level_breakdown"].iloc[0])
# add_clause_level_vectors(df["clause_level_breakdown"].iloc[0])

In [18]:
df["clause_vectors"] = df["clause_level_breakdown"].apply(lambda x: add_clause_level_vectors(x))

In [19]:
df.head(1)

,Review,Date,Stars,Restaurant,feature_descriptors,clause_level_breakdown,vector_descriptors,clause_vectors
0,There are lots of Indian restaurant at Tanjong...,4 days ago,5 stars,Gravy_Restaurant,"[{'lots': ['Indian'], 'restaurant': [], 'area'...",[{'lots': 'There are lots of Indian restaurant...,"[{'lots': [], 'restaurant': [], 'area': [[-0.0...",[{'lots': [[-0.51533 0.83186 0.22457 -0.7...


### Ground Truth Polarity

In [20]:
def get_word_polarity(feats):
  feats = feats[0]

  newdict = dict()
  for i in list(feats.keys()):
    vals = feats[i]
    polarity_list = []
    for val in vals:
      # Get sentiment
      doc = en(val)
      polarity_list.append(doc._.polarity)
    newdict[i] = polarity_list
  return [newdict]

In [21]:
print(df["feature_descriptors"].iloc[10][0])
print(get_word_polarity(df["feature_descriptors"].iloc[10])[0])

{'dish': ['friendly'], 'staff': ['really', 'appreciate', 'serving', 'yummy'], 'cottage': [], 'cheese': [], 'milk': ['flavoured', 'complimentary'], 'dessert': [], 'birthday': ['😊']}
{'dish': [0.375], 'staff': [0.2, 0.0, 0.0, 0.0], 'cottage': [], 'cheese': [], 'milk': [0.0, 0.3], 'dessert': [], 'birthday': [0.0]}


In [22]:
df["word_level_polarity"] = df["feature_descriptors"].apply(lambda x:get_word_polarity(x))

In [23]:
df.head(1)

,Review,Date,Stars,Restaurant,feature_descriptors,clause_level_breakdown,vector_descriptors,clause_vectors,word_level_polarity
0,There are lots of Indian restaurant at Tanjong...,4 days ago,5 stars,Gravy_Restaurant,"[{'lots': ['Indian'], 'restaurant': [], 'area'...",[{'lots': 'There are lots of Indian restaurant...,"[{'lots': [], 'restaurant': [], 'area': [[-0.0...",[{'lots': [[-0.51533 0.83186 0.22457 -0.7...,"[{'lots': [0.0], 'restaurant': [], 'area': [0...."


In [24]:
# df.to_csv("vector_output.csv")

### Feature Categorisation

In [25]:
# Use semeval2014 data:
# Category -> words
corpus = {
  'food': [],
  'service': [],
  'price': ['price','bargain','deal','money', 'value', 'cheap', 'expensive'],
  'ambience': [],
}

import pandas as pd
import xml.etree.ElementTree as ET

tree = ET.parse('Restaurants_2016.xml')
root = tree.getroot()

data = []
xml_col_val= dict()

for child in root:
  for sentences in child:
    for s in sentences:
      xml_col_val['id'] = s.attrib.get('id')
    for opinion in child.iter('Opinions'):
      target = []
      category = []
      for o in opinion:
        t = o.attrib['target']                 
        c = o.attrib['category']
        if t != "NULL" and "PRICE" not in c and "RESTAURANT" not in c:
          c = c.split('#')[0].lower()
          if c == 'drinks':
            c = 'food'
          elif c=='location':
            c = 'ambience'
          tl = t.lower().split()
          for tt in tl:
            if tt != 'price' and tt != 'deal':
              target.append(tt)
              category.append(c)
      xml_col_val['target'] = target
      xml_col_val['category'] = category   
    data.append(xml_col_val)
    xml_col_val = {}

# print(data)
data = pd.DataFrame(data).dropna()
data = data[data.category.map(len) != 0]
data = data.set_index(['id']).apply(pd.Series.explode).reset_index()
print(data)


            id    target  category
0    1041457:6     place  ambience
1    1041457:6     decor  ambience
2    1041457:6      vent  ambience
3    1053543:4   service   service
4    1064477:4      half      food
..         ...       ...       ...
307     WE#1:5     pasta      food
308    FF#6:12     staff   service
309    FF#6:12  customer   service
310    FF#6:12   service   service
311   FF#10:11     staff   service

[312 rows x 3 columns]


In [26]:
def updatecategories(c, t):
  curr = corpus[c]
  lt = t.split()
  for w in lt:
    if w not in curr:
      curr.append(w)
      corpus[c] = curr

data.apply(lambda x: updatecategories(x['category'], x['target']), axis=1)
# print(data)
# Words -> category
categories = {word: key for key, words in corpus.items() for word in words}
# print(categories)

# Embeddings for available words
data_embeddings = {key: value for key, value in embeddings_index.items() if key in categories.keys()}

# Processing the query
def process(query):
  qq = query.split()
  # print(qq)
  scores = {}
  food = corpus['food']
  service = corpus['service']
  ambience = corpus['ambience']
  price = corpus['price']
  for q in qq:
    if q in food:
      return 'food'
    elif q in price:
      return 'price'
    elif q in service:
      return 'service'
    elif q in ambience:
      return 'ambience'
    elif q in embeddings_index:
      query_embed = embeddings_index[q]
    else:
      return 'food'
    for word, embed in data_embeddings.items():
      category = categories[word]
      dist = query_embed.dot(embed)
      dist /= len(corpus[category])
      scores[category] = scores.get(category, 0) + dist
      # print(max(scores, key=scores.get))
  return max(scores, key=scores.get)

In [27]:
process("music")

'price'

In [28]:
def put_categories(feats):
  feats = feats[0]

  retdict = dict()
  for key in list(feats.keys()):
    output = process(key)
    retdict[key] = output
  return [retdict]

In [29]:
put_categories(df["feature_descriptors"].iloc[1])

[{'couple': 'price',
  'dal': 'food',
  'experience': 'price',
  'makhni': 'food',
  'place': 'ambience',
  'sugar': 'price',
  'times': 'price'}]

In [30]:
df["categorisation"] = df["feature_descriptors"].apply(lambda x:put_categories(x))

In [31]:
df.head(3)

,Review,Date,Stars,Restaurant,feature_descriptors,clause_level_breakdown,vector_descriptors,clause_vectors,word_level_polarity,categorisation
0,There are lots of Indian restaurant at Tanjong...,4 days ago,5 stars,Gravy_Restaurant,"[{'lots': ['Indian'], 'restaurant': [], 'area'...",[{'lots': 'There are lots of Indian restaurant...,"[{'lots': [], 'restaurant': [], 'area': [[-0.0...",[{'lots': [[-0.51533 0.83186 0.22457 -0.7...,"[{'lots': [0.0], 'restaurant': [], 'area': [0....","[{'lots': 'price', 'restaurant': 'ambience', '..."
2,"The dal makhni was sweet, seemed like sugar wa...",a week ago,2 stars,Gravy_Restaurant,"[{'dal': [], 'makhni': ['sweet', 'seemed'], 's...","[{'dal': 'The dal makhni was sweet', 'makhni':...","[{'dal': [], 'makhni': [[ 0.10788 0.38359 ...",[{'dal': [[-0.90375 -1.43 -0.023677 -0...,"[{'dal': [], 'makhni': [0.35, 0.0], 'sugar': [...","[{'dal': 'food', 'makhni': 'food', 'sugar': 'p..."
4,We are regular customers - the food is amazing...,2 weeks ago,5 stars,Gravy_Restaurant,"[{'customers': [], 'food': ['had']}]","[{'customers': 'We are regular customers', 'fo...","[{'customers': [], 'food': [[ 0.63256 -0.1271...",[{'customers': [[-0.51533 0.83186 0.22457 ...,"[{'customers': [], 'food': [0.0]}]","[{'customers': 'price', 'food': 'food'}]"


In [32]:
# df.to_csv("categorised.csv")

In [33]:
# with open('classifier_output.txt', 'w') as f:
#     for n in nouns:
#       f.write(f'{n}: {process(n)}')
#       f.write('\n')



print(corpus['food'])
#['half', 'sushi', 'lava', 'cake', 'dessert', 'food', 'fried', 'shrimp', 'tuna', 'of', 'gari', 'pie', 'lobster', 'teriyaki', 'rose', 'special', 'roll', 'scallion', 'pancakes', 'dumplings', 'chef', 'tartar', 'appetizer', 'wine', 'list', 'shabu-shabu', 'taxan', 'choices', 'eggs', 'benedict', 'portions', 'sandwich', 'thai', 'cuisine', 'caviar', 'crust', 'ingredients', 'premium', 'sake', 'menu', 'cold', 'dishes', 'sushimi', 'cucumber', 'turkey', 'burgers', 'japanese', 'pad', 'balsamic', 'vinegar', 'over', 'icecream', 'meal', 'se', 'ew', 'chicken', 'eats', 'bagels', 'measures', 'liquers', 'sassy', 'lassi', 'taiwanese', 'yakitori', '(bbq', 'meats)', 'pasta', 'penne', 'wines', 'selection', 'mussels', 'bbq', 'salmon', 'sea', 'bass', 'crispy', 'duck', 'pastas', 'risottos', 'sepia', 'braised', 'rabbit', 'dancing,', 'white', 'river', 'and', 'millenium', 'rolls', 'greek', 'cypriot', 'gyro', 'hot', 'dog', 'appetizers', 'modern', 'toppings', 'indian', 'chinese', 'farro', 'salad', 'mashed', 'yukon', 'potatoes', 'margherita', 'pizza', 'meat', 'sauces', 'korean', 'fusion', 'twists', 'pork', 'belly', 'tacos', 'dogs', 'croquette', 'bun', 'flank', 'steak']
print(corpus['price'])
#['price', 'bargain', 'deal', 'money', 'value', 'cheap', 'expensive']
print(corpus['ambience'])
#['place', 'decor', 'vent', 'ambience', 'setting', 'downstairs', 'lounge', 'traditional', 'french', 'decour', 'hall', 'semi-private', 'boths', 'atmosphere', 'restaurant', 'ambiance', 'unisex', 'bathroom', 'blond', 'wood', 'room', 'terrace', 'bar', 'scene', 'patio', 'rice', 'avenue', 'view', 'of', 'river', 'and', 'nyc', 'hookah', 'mirrors', 'zenkichi', 'fire']
print(corpus['service'])
#['service', 'wait', 'staff', 'clerks', 'management', 'wait-staff', 'delivery', 'guys', 'guy', 'survice', 'people', 'servers', 'customer']


['half', 'sushi', 'lava', 'cake', 'dessert', 'food', 'fried', 'shrimp', 'tuna', 'of', 'gari', 'pie', 'lobster', 'teriyaki', 'rose', 'special', 'roll', 'scallion', 'pancakes', 'dumplings', 'chef', 'tartar', 'appetizer', 'wine', 'list', 'shabu-shabu', 'taxan', 'choices', 'eggs', 'benedict', 'portions', 'sandwich', 'thai', 'cuisine', 'caviar', 'crust', 'ingredients', 'premium', 'sake', 'menu', 'cold', 'dishes', 'sushimi', 'cucumber', 'turkey', 'burgers', 'japanese', 'pad', 'balsamic', 'vinegar', 'over', 'icecream', 'meal', 'se', 'ew', 'chicken', 'eats', 'bagels', 'measures', 'liquers', 'sassy', 'lassi', 'taiwanese', 'yakitori', '(bbq', 'meats)', 'pasta', 'penne', 'wines', 'selection', 'mussels', 'bbq', 'salmon', 'sea', 'bass', 'crispy', 'duck', 'pastas', 'risottos', 'sepia', 'braised', 'rabbit', 'dancing,', 'white', 'river', 'and', 'millenium', 'rolls', 'greek', 'cypriot', 'gyro', 'hot', 'dog', 'appetizers', 'modern', 'toppings', 'indian', 'chinese', 'farro', 'salad', 'mashed', 'yukon', '

### Sentiment Analysis

In [34]:
def clause_level_polarity(feats):
  retdict = dict()
  feats = feats[0]
  for feature in list(feats.keys()):

    doc = en(feats[feature])
    retdict[feature] = doc._.polarity
    # print(doc._.polarity)
  return [retdict]

In [35]:
clause_level_polarity(df["clause_level_breakdown"].iloc[1])

[{'couple': 0.0,
  'dal': 0.35,
  'experience': -0.3499999999999999,
  'makhni': 0.35,
  'place': 0.0,
  'sugar': 0.0,
  'times': 0.0}]

In [36]:
df["clause_polarity"] = df["clause_level_breakdown"].apply(lambda x: clause_level_polarity(x))

In [37]:
df.head(3)

,Review,Date,Stars,Restaurant,feature_descriptors,clause_level_breakdown,vector_descriptors,clause_vectors,word_level_polarity,categorisation,clause_polarity
0,There are lots of Indian restaurant at Tanjong...,4 days ago,5 stars,Gravy_Restaurant,"[{'lots': ['Indian'], 'restaurant': [], 'area'...",[{'lots': 'There are lots of Indian restaurant...,"[{'lots': [], 'restaurant': [], 'area': [[-0.0...",[{'lots': [[-0.51533 0.83186 0.22457 -0.7...,"[{'lots': [0.0], 'restaurant': [], 'area': [0....","[{'lots': 'price', 'restaurant': 'ambience', '...","[{'lots': 0.0, 'restaurant': 0.0, 'time': 0.0,..."
2,"The dal makhni was sweet, seemed like sugar wa...",a week ago,2 stars,Gravy_Restaurant,"[{'dal': [], 'makhni': ['sweet', 'seemed'], 's...","[{'dal': 'The dal makhni was sweet', 'makhni':...","[{'dal': [], 'makhni': [[ 0.10788 0.38359 ...",[{'dal': [[-0.90375 -1.43 -0.023677 -0...,"[{'dal': [], 'makhni': [0.35, 0.0], 'sugar': [...","[{'dal': 'food', 'makhni': 'food', 'sugar': 'p...","[{'dal': 0.35, 'makhni': 0.35, 'couple': 0.0, ..."
4,We are regular customers - the food is amazing...,2 weeks ago,5 stars,Gravy_Restaurant,"[{'customers': [], 'food': ['had']}]","[{'customers': 'We are regular customers', 'fo...","[{'customers': [], 'food': [[ 0.63256 -0.1271...",[{'customers': [[-0.51533 0.83186 0.22457 ...,"[{'customers': [], 'food': [0.0]}]","[{'customers': 'price', 'food': 'food'}]","[{'customers': 0.0, 'food': 0.6000000000000001}]"


In [38]:
df.columns

Index(['Review', 'Date', 'Stars', 'Restaurant', 'feature_descriptors',
       'clause_level_breakdown', 'vector_descriptors', 'clause_vectors',
       'word_level_polarity', 'categorisation', 'clause_polarity'],
      dtype='object')

In [39]:
df = df[['Review', 'Date', 'Stars', 'Restaurant','feature_descriptors', 'vector_descriptors','word_level_polarity', 'clause_level_breakdown', 'clause_vectors', 'clause_polarity', 'categorisation']]

In [40]:
df.head(3)

,Review,Date,Stars,Restaurant,feature_descriptors,vector_descriptors,word_level_polarity,clause_level_breakdown,clause_vectors,clause_polarity,categorisation
0,There are lots of Indian restaurant at Tanjong...,4 days ago,5 stars,Gravy_Restaurant,"[{'lots': ['Indian'], 'restaurant': [], 'area'...","[{'lots': [], 'restaurant': [], 'area': [[-0.0...","[{'lots': [0.0], 'restaurant': [], 'area': [0....",[{'lots': 'There are lots of Indian restaurant...,[{'lots': [[-0.51533 0.83186 0.22457 -0.7...,"[{'lots': 0.0, 'restaurant': 0.0, 'time': 0.0,...","[{'lots': 'price', 'restaurant': 'ambience', '..."
2,"The dal makhni was sweet, seemed like sugar wa...",a week ago,2 stars,Gravy_Restaurant,"[{'dal': [], 'makhni': ['sweet', 'seemed'], 's...","[{'dal': [], 'makhni': [[ 0.10788 0.38359 ...","[{'dal': [], 'makhni': [0.35, 0.0], 'sugar': [...","[{'dal': 'The dal makhni was sweet', 'makhni':...",[{'dal': [[-0.90375 -1.43 -0.023677 -0...,"[{'dal': 0.35, 'makhni': 0.35, 'couple': 0.0, ...","[{'dal': 'food', 'makhni': 'food', 'sugar': 'p..."
4,We are regular customers - the food is amazing...,2 weeks ago,5 stars,Gravy_Restaurant,"[{'customers': [], 'food': ['had']}]","[{'customers': [], 'food': [[ 0.63256 -0.1271...","[{'customers': [], 'food': [0.0]}]","[{'customers': 'We are regular customers', 'fo...",[{'customers': [[-0.51533 0.83186 0.22457 ...,"[{'customers': 0.0, 'food': 0.6000000000000001}]","[{'customers': 'price', 'food': 'food'}]"


In [41]:
df.to_pickle("testing_final_processed.pkl")

### Break dataset into train, validate and test

In [42]:
train, validate, test = \
              np.split(df.sample(frac=1, random_state=42), 
                       [int(.6*len(df)), int(.8*len(df))])

In [43]:
print(len(train), len(validate), len(test))

814 271 272


In [44]:
train.to_pickle("train.pkl")
validate.to_pickle("validate.pkl")
test.to_pickle("test.pkl")

In [45]:
def get_table(df):
  retdf = pd.DataFrame()
  
  for r in range(0, len(df)):
    row = df.iloc[r, :]# Get entire row
    # print(row)
    # break
    count = 0
    for feature in row["clause_vectors"][0].keys():
      rowdict = dict()
      rowdict["restaurant"] = row["Restaurant"]
      rowdict["stars"] = row["Stars"]
      rowdict["date"] = row["Date"]
      rowdict["feat_index"] = count
      count += 1
      # print("review_index", row.iloc[0])
      rowdict["index"] = row.iloc[0]
      rowdict["feature"] = feature
      try:
        rowdict["category"] = row["categorisation"][0][feature]
      except:
        rowdict["category"] = "Non-Categorised"
      # print(feature, row["clause_level_breakdown"][0][feature], '\n')
      rowdict["raw_clause"] = row["clause_level_breakdown"][0][feature]
      # print(feature, row["clause_vectors"][0][feature], '\n')
      rowdict["clause_vector"] = [row["clause_vectors"][0][feature]]
      # print(feature, row["clause_polarity"][0][feature], '\n')
      rowdict["clause_polarity"] = row["clause_polarity"][0][feature]
      
      # print(row["clause_vectors"][0][feature])
        
      # Make df row 
      newrow = pd.DataFrame.from_dict(rowdict, orient="columns")
      retdf = retdf.append(newrow)
      # return newrow
    # break
  return retdf

sentdf = get_table(df)
sentdf.head(1)

,restaurant,stars,date,feat_index,index,feature,category,raw_clause,clause_vector,clause_polarity
0,Gravy_Restaurant,5 stars,4 days ago,0,There are lots of Indian restaurant at Tanjong...,lots,price,There are lots of Indian restaurant at Tanjong...,"[[-0.51533, 0.83186, 0.22457, -0.73865, 0.1871...",0.0


In [46]:
sentdf.to_pickle("reg_sent_df.pkl")